In [1]:
import re
import json
import unicodedata
import mysql.connector
import numpy as np
import pandas as pd

In [2]:
def padSequence(feature, max_feature):
    pad = np.zeros(max_feature-len(feature), dtype=np.float32)
    feature = np.concatenate([feature, pad])
    return feature

In [3]:
def getWordEmbedding(word, cursor):
#     word = word.replace("'", "''")
    sql = """select vec from term where term like %s"""
    cursor.execute(sql, (str(word),))
    data = cursor.fetchall()
    if len(data) > 0:
        decoded_vec = json.JSONDecoder().decode(data[0][0])
        vec = np.asarray(decoded_vec, dtype=np.float32)
        return True, vec
    else:
        return False, data

In [4]:
def myTokenizer(content, lower=True):
    raw = content.split(' ')
    remover = re.compile("[^a-zA-Z-]")
    
    token = []
    
    for i in raw:
        term = remover.sub('', i)
        if lower == True:
            term = term.lower()
        token.append(term)
    tokenized = filter(None, token)
    
    return tokenized

In [5]:
def arrangeInputMatrix(data, labels):
    ftr_list = []
    cls_list = []
    
    unique_labels = pd.unique(labels)
    total_labels = len(unique_labels)
    
    for i in range(len(data)):
        cls = np.zeros((total_labels,), dtype=int)
        cls_idx = np.where(unique_labels == labels[i])[0][0]
        cls[cls_idx] = 1
        
        string = data[i].replace('\n', '')
        string = np.array(myTokenizer(string))

        begin = True
        for word in string:
            stat, vec = getWordEmbedding(word, cursor)
            if not stat:
                continue
            if begin:
                begin = False
                feature = vec
            else:
                feature += vec
                # feature = np.concatenate([feature, vec])

        feature = feature/np.linalg.norm(feature)
        ftr_list.append(feature)
        cls_list.append(cls)
        
    return np.array(ftr_list), np.array(cls_list)

In [6]:
print "convert into numpy array"
df = pd.read_csv("schematics/data_target/data_target.csv", sep=',', names=["documents", "labels"])
documents = df["documents"].values
labels = df["labels"].values

convert into numpy array


In [7]:
db = mysql.connector.connect(user="root", password='namamuran', database="glove")
cursor = db.cursor(buffered=True)
ftr, cls = arrangeInputMatrix(documents, labels)

In [8]:
print "checking len of array"
print len(documents) == len(ftr)
print len(documents) == len(cls)

checking len of array
True
True


In [9]:
np.save("output/data-target-schematics/ftr-data-target.npy", ftr)
np.save("output/data-target-schematics/cls-data-target.npy", cls)